In [1]:
#https://www.tensorflow.org/programmers_guide/variables
import tensorflow as tf
import numpy as np

In [ ]:
#全是关于variable的
my_variable = tf.get_variable('my_variable' , [1,2,3])#provide the name and shape
#上面默认的dtype为tf.float32
#而且默认值 是从tf.glorot_uniform_initializer 这里随机出来的

In [2]:
#也可以指定 dtype and initializer
#initializer 初始化器相当于函数函数
my_int_variable = tf.get_variable('my_int_variable',
                                 [1,2,3],
                                 dtype=tf.int32,
                                 initializer = tf.zeros_initializer)

In [3]:
#tensorflow提供了许多初始化器
#还可以使用tensor来初始化variable
#当初始化器为tf.Tensor时 就不需要指出shape了 由tensor给定
other_variable = tf.get_variable('other_variable',
                                dtype = tf.int32,
                                initializer = tf.constant([23,42]))

In [4]:
'''
Because disconnected parts of a TensorFlow program might want to create variables, 
it is sometimes useful to have a single way to access all of them. 
For this reason TensorFlow provides collections, 
which are named lists of tensors or other objects, such as tf.Variable instances.  
By default every tf.Variable gets placed in the following two collections: 
* tf.GraphKeys.GLOBAL_VARIABLES --- variables that can be shared across multiple devices, 
* tf.GraphKeys.TRAINABLE_VARIABLES--- variables for which TensorFlow will calculate gradients.  
If you don't want a variable to be trainable, 
add it to the tf.GraphKeys.LOCAL_VARIABLES collection instead. For example, 
the following snippet demonstrates how to add a variable named my_local to this collection:
'''
#下面代码 my_local变量 就不会被优化了
my_local = tf.get_variable('my_local',
                          shape=[1,2,3],
                          collections = [tf.GraphKeys.LOCAL_VARIABLES])
'''
Alternatively, you can specify trainable=False as an argument to tf.get_variable:
'''
my_non_trainable = tf.get_variable('my_non_trainable',
                                  shape=[1,2,3],
                                  trainable = False)

In [5]:
'''
You can also use your own collections. 
Any string is a valid collection name, 
and there is no need to explicitly create a collection. 
To add a variable (or any other object) to a collection after creating the variable, 
call tf.add_to_collection.
For example, the following code adds an existing variable 
    named my_local to a collection named my_collection_name:
'''
tf.add_to_collection('my_collection_name' , my_local)
'''
And to retrieve a list of all the variables (or other objects) 
    you've placed in a collection you can use:
'''
tf.get_collection('my_collection_name')

[<tf.Variable 'my_local:0' shape=(1, 2, 3) dtype=float32_ref>]

In [6]:
tf.get_collection(tf.GraphKeys.VARIABLES)

[<tf.Variable 'my_int_variable:0' shape=(1, 2, 3) dtype=int32_ref>,
 <tf.Variable 'other_variable:0' shape=(2,) dtype=int32_ref>,
 <tf.Variable 'my_non_trainable:0' shape=(1, 2, 3) dtype=float32_ref>]

In [13]:
#=========
#=========
#设备部分
#可以将variable放在指定的设备上面
#将variable v 放在第二个GPU上
with tf.device('/device:GPU:1'):
    v = tf.get_variable('v' , [1])


ValueError: Variable v already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "I:\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1204, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "I:\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 2630, in create_op
    original_op=self._default_original_op, op_def=op_def)
  File "I:\Anaconda3\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 767, in apply_op
    op_def=op_def)


In [14]:
'''
It is particularly important for variables to be in the correct device in distributed settings. 
Accidentally putting variables on workers instead of parameter servers, 
for example, can severely slow down training or, in the worst case, 
let each worker blithely forge ahead with its own independent copy of each variable. 
For this reason we provide tf.train.replica_device_setter, which can automatically place variables in parameter servers. For example:
'''
cluster_spec = {
    "ps": ["ps0:2222", "ps1:2222"],
    "worker": ["worker0:2222", "worker1:2222", "worker2:2222"]}
with tf.device(tf.train.replica_device_setter(cluster=cluster_spec)):
    v = tf.get_variable("v", shape=[20, 20])  # this variable is placed 
                                            # in the parameter server
                                            # by the replica_device_setter

ValueError: Variable v already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "I:\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1204, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "I:\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 2630, in create_op
    original_op=self._default_original_op, op_def=op_def)
  File "I:\Anaconda3\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 767, in apply_op
    op_def=op_def)


In [ ]:
#接下来为变量的初始化 变量必须经过初始化才能使用 明确地进行初始化
#创建自己的graph session 使用低级API
#高级的框架 例如tf.contrib.slim, tf.estimator.Estimator and Keras
#在训练一个模型之前自动初始化variables
'''
明确的初始化 也有一定的好处
Explicit initialization is otherwise useful 
because it allows you not to rerun potentially expensive initializers
when reloading a model from a checkpoint as well as allowing determinism 
when randomly-initialized variables are shared in a distributed setting.
'''

In [ ]:
#初始化所有的trainable变量
#tf.global_variables_initializer()
#这个函数返回一个操作负责初始化在tf.GraphKeys.GLOBAL_VARIABLES collection中的所有变量

#sess.run(tf.global_variables_initializer())

In [ ]:
#也可以使用variable自己的初始化器（就是在get_variable函数中的那个初始化器）

#sess.run(my_varlable.initializer)

In [15]:
#也可以查看哪些变量没有被初始化
with tf.Session() as sess:
    print(sess.run(tf.report_uninitialized_variables()))

InvalidArgumentError: Cannot assign a device for operation 'v': Operation was explicitly assigned to /device:GPU:1 but available devices are [ /job:localhost/replica:0/task:0/cpu:0 ]. Make sure the device specification refers to a valid device.
	 [[Node: v = VariableV2[_class=["loc:@v"], container="", dtype=DT_FLOAT, shape=[1], shared_name="", _device="/device:GPU:1"]()]]

Caused by op 'v', defined at:
  File "I:\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "I:\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "I:\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "I:\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "I:\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "I:\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.p

In [20]:
#默认的tf.global_variables_initializer()
#不能指定变量的初始化过程 所以当一个变量以来另一个变量的时候
#会发生错误
#任何时候 在上下文中使用一个变量的值 并不是所有的变量已经初始化
#上面的意思就是 使用一个变量的值初始化别的变量
#所以如果想用一个变量初始化另一个变量时候 应该像下面一样书写 而不是使用变量进行初始化
#initialized_value 返回刚刚初始化后的值

ccc = tf.get_variable('ccc' , shape=[1,2],initializer=tf.zeros_initializer())
bb = tf.get_variable('bb' , initializer=ccc.initialized_value() + 1)

In [ ]:
#如何使用 variables
vv = tf.get_variable('vv' , shape=[1,2] , initializer=tf.zeros_initializer())
ww = vv + 1

In [4]:
#可以使用variable的成员函数 assign将一个值分配给variable
#assign assign_add函数都可以
with tf.Session() as sess:
    v4 = tf.get_variable('v4' , shape=() , initializer=tf.zeros_initializer())
    assignment = v4.assign_add(1)
    tf.global_variables_initializer().run()
    sess.run(assignment)
    
    print(sess.run(v4))

1.0


In [15]:
'''
Because variables are mutable it's sometimes useful to know what version of a variable's value 
is being used at any point in time. To force a re-read of the value of a variable after something has happened, 
you can use tf.Variable.read_value. For example:

'''

#可以追踪变量 在一些事情发生之后 进行追踪
v33 = tf.get_variable('v33' , shape=() , initializer=tf.zeros_initializer())


In [8]:
assignment2 = v2.assign_add(1)

with tf.control_dependencies([assignment2]):
    w = v2.read_value()

In [9]:
w

<tf.Tensor 'read:0' shape=() dtype=float32>

In [18]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    haha = v33.assign(2)
    sess.run(haha)
    #可以将v33的值读出来 看看 进行跟踪
    ww = v33.read_value()
    print(sess.run(ww))

2.0


In [2]:
#tensorflow的 变量 共享
def conv_relu(inputt , kernel_shape , bias_shape):
    weights = tf.get_variable('weight' , kernel_shape,
                             initializer = tf.random_normal_initializer())
    biasese = tf.get_variable('biases' , bias_shape,
                             initializer = tf.constant_initializer(0.0))
    
    conv = tf.nn.conv2d(inputt , weights , strides=[1,1,1,1] , padding='SAME')
    
    return tf.nn.relu(conv + biasese)

#上面的这个函数 重复使用就会出现错误 因为get_variable认为出现了重复的变量


In [3]:
input1 = tf.random_normal([1,10,10,32])
input2 = tf.random_normal([1,20,20,32])
x = conv_relu(input1, kernel_shape=[5, 5, 32, 32], bias_shape=[32])
x = conv_relu(x, kernel_shape=[5, 5, 32, 32], bias_shape = [32])  # This fails.

ValueError: Variable weight already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "I:\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1204, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "I:\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 2630, in create_op
    original_op=self._default_original_op, op_def=op_def)
  File "I:\Anaconda3\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 767, in apply_op
    op_def=op_def)


In [4]:
#Calling conv_relu in different scopes clarifies that we want to create new variables:
def my_image_filter(input_images):
    with tf.variable_scope('conv1'):
        # Variables created here will be named "conv1/weights", "conv1/biases"
        relu1 = conv_relu(input_images , [5,5,32,32] , [32])
    
    with tf.variable_scope('conv2'):
        # Variables created here will be named "conv2/weights", "conv2/biases"
        return conv_relu(relu1 , [5,5,32,32] , [32])

In [5]:
#如果想共享变量 有两种方式
#第一种 可以使用reuse=True 来创建一个名字相同的scope
with tf.variable_scope('model1'):
    output1 = my_image_filter(input1)
with tf.variable_scope('model' , reuse=True):
    output2 = my_image_filter(input2)

#第二种方式 使用函数 scope.reuse_variables()
with tf.variable_scope('model') as scope:
    output1 = my_image_filter(input1)
    scope.reuse_variables()
    output2 = my_image_filter(input2)

ValueError: Variable model/conv1/weight does not exist, or was not created with tf.get_variable(). Did you mean to set reuse=None in VarScope?

In [6]:
#由于使用范围的精确字符串名称可能会感到危险，因此也可以根据另一个变量来初始化一个变量范围
#Since depending on exact string names of scopes can feel dangerous, it's also possible to initialize a variable scope based on another one:
with tf.variable_scope('model') as scope:
    output1 = my_image_filter(input1)
with tf.variable_scope(scope , reuse=True):
    output2 = my_image_filter(input2)